## TTS module for our Pipeline

Dans ce notebook on va générer des fichiers audios à partir de textes.

On va premièrement utiliser le modèle [gTTS](https://gtts.readthedocs.io/en/latest/) qui est un modèle de TTS (Text To Speech) basé sur Google Translate.
Sur ce modèle nous allons utiliser le dataset [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) qui est un dataset de textes en anglais. Celui ci nous permettra d'évaluer la qualité de notre modèle.

Ensuite, nous utilisons le dataset des transcripts de sortie du GEC. Cette partie simulera l'utilisation de notre modèle dans la pipeline, et pourra être réutilisée directement dans la pipeline si les résultats sont considérés comme satisfaisants.


Autres modèles de TTS :
- [Tacotron2](https://huggingface.co/transformers/model_doc/tacotron2.html) (exemple avec vocodeur HiFiGAN, sur modèle pré-entrainé avec le dataset LJSpeech [ici](https://huggingface.co/speechbrain/tts-tacotron2-ljspeech))
- [FastSpeech2](https://fastspeech2.github.io/fastspeech2/)
Vocodeurs:
- [HiFiGAN](https://pytorch.org/hub/nvidia_deeplearningexamples_hifigan/)
- [MelGAN](https://paperswithcode.com/method/melgan)


Les datasets se trouvent dans un dossier `data/` et les fichiers audios générés dans un dossier `output/`.

### Import des librairies

In [2]:
import torchaudio, torch, os
import IPython.display as ipd
from torch.utils.data import Dataset, DataLoader

from gtts import gTTS
from tqdm.notebook import tqdm

## Définition des fonctions

In [9]:
class GEC_Dataset(Dataset):
    def __init__(self, path):
        self.path = path
        self.files = os.listdir(path)
        self.sentences = []
        for file in self.files:
            with open(path + '\\' + file, 'r', encoding='utf-8') as file:
                for line in file:
                    self.sentences.append(line.strip())
    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        transcript = self.sentences[idx]

        return transcript, idx


def load_librispeech_dataset(path='data\\'):
    librispeech_dataset = torchaudio.datasets.LIBRISPEECH(path, url='dev-clean', download=True)
    #Shorten dataset to 500 samples
    librispeech_dataset_short = torch.utils.data.Subset(librispeech_dataset, range(500))
    librispeech_dataloader = DataLoader(librispeech_dataset_short, batch_size=1)
    
    print("Number of samples:", len(librispeech_dataset_short))
    
    sample_meta0 = librispeech_dataset.get_metadata(0)
    sample_0 = librispeech_dataset_short[0][0]
    print("""
    Audio n°0 :
    
    Path audio: {}{}
    Sample rate : {}
    Transcript : {}
    Speaker ID : {}
    Chapter ID : {}
    Utterance ID : {}
    """.format(path + "LibriSpeech", *sample_meta0))
    
    #Playing sample_O
    ipd.Audio(sample_0, rate=sample_meta0[1])
    
    return librispeech_dataloader

def process_librispeech(data, model='gtts', output_folder='output'):
    (waveform, sample_rate, transcript, ID_s, ID_c, ID_u) = data
    
    waveform = waveform.squeeze(0)
    sample_rate = sample_rate.squeeze(0)
    transcript = str(transcript[0])
    ID = "-".join((str(ID_s.item()), str(ID_c.item()), str(ID_u.item()))).replace(',','')
    
    file = f'./{output_folder}/out_{ID}.mp3'
    
    if os.path.exists(file):
        return ID, file, 1, _
    
    return ID, file, 0, transcript


def load_GEC_dataset(path='data\\GEC', dataset=GEC_Dataset):
    GEC_Dataset = dataset(path)
    GEC_Dataloader = DataLoader(GEC_Dataset, batch_size=1)
    
    print("Number of samples:", len(GEC_Dataset))
    
    sample_meta0 = GEC_Dataloader.dataset[0]
    print("""
    Transcript n°0 :
    
    Transcript : {}
    Transcript ID : {}
    """.format(*sample_meta0))
    
    return GEC_Dataloader


def process_GEC(data, model='gtts', output_folder='output'):
    (transcript, ID) = data
    ID = ID.item()
    file = f'./{output_folder}/out_{ID}.mp3'
    
    if os.path.exists(file):
        return ID, file, 1, _
    
    return ID, file, 0, transcript

In [10]:
def TTS(model='gtts', output_folder='output', dataset='librispeech'):
    output_folder = output_folder + '/' + model + '_' + dataset
    os.makedirs(output_folder, exist_ok=True)
    # TQDM loader
    try:
        function = "load_" + dataset + "_dataset"
        dataloader = eval(function)()
        dataloader_tqdm = tqdm(dataloader, total=len(dataloader))
    
        process_function = "process_" + dataset
        
    except NameError:
        raise ValueError(f'Unknown dataset {dataset}')
    
    print(f'Generating audio files with {model} model in {output_folder} folder')
    
    for i, data in enumerate(dataloader_tqdm):
        (ID, file_out, out, transcript) = eval(process_function)(data, model=model, output_folder=output_folder)
        file_out = file_out if type(file_out) == str else file_out[0]
        transcript = str(transcript)
        if out == 1:
            dataloader_tqdm.set_postfix({'status': 'Skipped', 'ID': ID})
            continue

        try:
            #Passage du transcript dans le modèle
            if model == 'gtts':
                tts = gTTS(transcript, lang='en', tld='co.in')
                tts.save(file_out)
            elif model == 'tacotron2':
                pass
            else:
                raise ValueError(f'Unknown model {model}')
            
            dataloader_tqdm.set_postfix()

        except Exception as e:
            dataloader_tqdm.set_postfix({'status': 'Error', 'ID': ID})
            raise e
            
        
        #Vérification
        # Comparer les fichiers audios générés avec les fichiers audios du dataset (spectrogrammes ?)

    

___
## TTS model gtts
### Dataset Librispeech
Le même qu'on a utilisé pour générer les fichiers audios pour le dataset

In [7]:
TTS(model='gtts', output_folder='output', dataset='librispeech')

Number of samples: 500

    Audio n°0 :
    
    Path audio: data\LibriSpeechdev-clean\1272\128104\1272-128104-0000.flac
    Sample rate : 16000
    Transcript : MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL
    Speaker ID : 1272
    Chapter ID : 128104
    Utterance ID : 0
    


  0%|          | 0/500 [00:00<?, ?it/s]

Generating audio files with gtts model in output/gtts_librispeech folder


#### Dataset GEC (pour pipeline)

In [11]:
TTS(model="gtts", output_folder="output", dataset="GEC")

Number of samples: 17

    Transcript n°0 :
    
    Transcript : She doesn't like to eat vegetables.
    Transcript ID : 0
    


  0%|          | 0/17 [00:00<?, ?it/s]

Generating audio files with gtts model in output/gtts_GEC folder
